In [2]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
from matplotlib import pyplot as plt

In [3]:
epoch = 20
batch_size = 100
lr = 0.005

In [4]:
train_data = torchvision.datasets.MNIST(
    root='./mnist/',
    train=True,                                    
    transform=torchvision.transforms.ToTensor(),   
    download=True,                       
)

In [5]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.Tanh(),
            nn.Linear(128, 64),
            nn.Tanh(),
            nn.Linear(64, 12),
            nn.Tanh(),
            nn.Linear(12, 3),   
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.Tanh(),
            nn.Linear(12, 64),
            nn.Tanh(),
            nn.Linear(64, 128),
            nn.Tanh(),
            nn.Linear(128, 28*28),
            nn.Sigmoid(),       # compress to a range (0, 1)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
autoencoder = AutoEncoder()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=lr)
loss_func = nn.MSELoss()

In [7]:
for epoch in range(epoch):
    for step, (x, b_label) in enumerate(train_loader):
        b_x = x.view(-1, 28*28)
        b_y = x.view(-1, 28*28)

        encoded, decoded = autoencoder(b_x)

        loss = loss_func(decoded, b_y)      # mean square error
        optimizer.zero_grad()               # clear gradients for this training step
        loss.backward()                     # backpropagation, compute gradients
        optimizer.step()                    # apply gradients

        if step % 1000 == 0:
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.item())

Epoch:  0 | train loss: 0.2314
Epoch:  1 | train loss: 0.0488
Epoch:  2 | train loss: 0.0376
Epoch:  3 | train loss: 0.0341
Epoch:  4 | train loss: 0.0382
Epoch:  5 | train loss: 0.0330
Epoch:  6 | train loss: 0.0328
Epoch:  7 | train loss: 0.0369
Epoch:  8 | train loss: 0.0341
Epoch:  9 | train loss: 0.0322
Epoch:  10 | train loss: 0.0338
Epoch:  11 | train loss: 0.0306
Epoch:  12 | train loss: 0.0328
Epoch:  13 | train loss: 0.0353
Epoch:  14 | train loss: 0.0342
Epoch:  15 | train loss: 0.0312
Epoch:  16 | train loss: 0.0372
Epoch:  17 | train loss: 0.0361
Epoch:  18 | train loss: 0.0319
Epoch:  19 | train loss: 0.0339


In [9]:
import os
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), 
            nn.Linear(64, 12), 
            nn.ReLU(True), 
            nn.Linear(12, 8))
        self.decoder = nn.Sequential(
            nn.Linear(8, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), 
            nn.Linear(128, 28 * 28), 
            nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')

epoch [1/100], loss:0.1904
epoch [2/100], loss:0.1625
epoch [3/100], loss:0.1384
epoch [4/100], loss:0.1352
epoch [5/100], loss:0.1182
epoch [6/100], loss:0.1122
epoch [7/100], loss:0.1184
epoch [8/100], loss:0.1199
epoch [9/100], loss:0.1172
epoch [10/100], loss:0.1132
epoch [11/100], loss:0.1079
epoch [12/100], loss:0.1044
epoch [13/100], loss:0.1133
epoch [14/100], loss:0.1031
epoch [15/100], loss:0.1082
epoch [16/100], loss:0.0931
epoch [17/100], loss:0.1032
epoch [18/100], loss:0.0991
epoch [19/100], loss:0.1012
epoch [20/100], loss:0.0943
epoch [21/100], loss:0.0912
epoch [22/100], loss:0.0886
epoch [23/100], loss:0.0859
epoch [24/100], loss:0.0936
epoch [25/100], loss:0.0955
epoch [26/100], loss:0.0767
epoch [27/100], loss:0.0815
epoch [28/100], loss:0.0959
epoch [29/100], loss:0.0874
epoch [30/100], loss:0.0955
epoch [31/100], loss:0.0824
epoch [32/100], loss:0.0822
epoch [33/100], loss:0.0872
epoch [34/100], loss:0.0801
epoch [35/100], loss:0.0804
epoch [36/100], loss:0.0904
e

In [10]:
import os
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')
    
if not os.path.exists('./filters'):
    os.mkdir('./filters')

def to_img(x):
    x = x.view(x.size(0), 1, 28, 28)
    return x

num_epochs = 20
batch_size = 128
learning_rate = 1e-3


def add_noise(img):
    noise = torch.randn(img.size()) * 0.2
    noisy_img = img + noise
    return noisy_img


def plot_sample_img(img, name):
    img = img.view(1, 28, 28)
    save_image(img, './sample_{}.png'.format(name))


def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor


def tensor_round(tensor):
    return torch.round(tensor)


img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor:min_max_normalization(tensor, 0, 1)),
    transforms.Lambda(lambda tensor:tensor_round(tensor))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 256),
            nn.ReLU(True),
            nn.Linear(256, 64),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(True),
            nn.Linear(256, 28 * 28),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder().cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        noisy_img = add_noise(img)
        noisy_img = Variable(noisy_img).cuda()
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(noisy_img)
        loss = criterion(output, img)
        MSE_loss = nn.MSELoss()(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data.item(), MSE_loss.data.item()))
    if epoch % 10 == 0:
        x = to_img(img.cpu().data)
        x_hat = to_img(output.cpu().data)
        x_noisy = to_img(noisy_img.cpu().data)
        weights = to_img(model.encoder[0].weight.cpu().data)
        save_image(x, './mlp_img/x_{}.png'.format(epoch))
        save_image(x_hat, './mlp_img/x_hat_{}.png'.format(epoch))
        save_image(x_noisy, './mlp_img/x_noisy_{}.png'.format(epoch))
        save_image(weights, './filters/epoch_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')

epoch [1/20], loss:0.1251, MSE_loss:0.0386
epoch [2/20], loss:0.0905, MSE_loss:0.0276
epoch [3/20], loss:0.0772, MSE_loss:0.0232
epoch [4/20], loss:0.0720, MSE_loss:0.0217
epoch [5/20], loss:0.0608, MSE_loss:0.0182
epoch [6/20], loss:0.0611, MSE_loss:0.0184
epoch [7/20], loss:0.0627, MSE_loss:0.0185
epoch [8/20], loss:0.0588, MSE_loss:0.0176
epoch [9/20], loss:0.0555, MSE_loss:0.0166
epoch [10/20], loss:0.0565, MSE_loss:0.0169
epoch [11/20], loss:0.0507, MSE_loss:0.0151
epoch [12/20], loss:0.0491, MSE_loss:0.0145
epoch [13/20], loss:0.0508, MSE_loss:0.0150
epoch [14/20], loss:0.0487, MSE_loss:0.0144
epoch [15/20], loss:0.0470, MSE_loss:0.0140
epoch [16/20], loss:0.0457, MSE_loss:0.0135
epoch [17/20], loss:0.0461, MSE_loss:0.0138
epoch [18/20], loss:0.0467, MSE_loss:0.0138
epoch [19/20], loss:0.0477, MSE_loss:0.0141
epoch [20/20], loss:0.0482, MSE_loss:0.0143
